In [23]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [24]:
import math
import torch

from atomics import Identity, Linear, ReLU

from tqdm.auto import trange

In [28]:
init_lr_list = 2.0**np.arange(-4,4)
# width_list = [64, 128, 256, 512, 1024, 2048]
depth_list = [2, 4, 8, 16, 32]
# wd_list = [0.0001, 0.001, 0.01, 0.1]
wd = 0.001

batch_size = 400
steps = 1000
beta = 0.9

x = torch.randn(batch_size, 8).cuda()
y = torch.randn(batch_size, 8).cuda()

results = {}

for depth in depth_list:
    results[depth] = []
    
    block = (1-1/depth) * Identity() + 1/depth * Linear(width, width) @ (math.sqrt(2) * ReLU())
    # block = Linear(width, width) @ (math.sqrt(2) * ReLU())
    net = Linear(8,width) @ block ** depth @ Linear(width, 8)
    
    for init_lr in init_lr_list:
    
        net.initialize()
        
        net = net.cuda()
        
        for i in (pbar := trange(steps)):
            out = net.forward(x)
            loss = (out-y).square().mean()
            loss.backward()
        
            net.update(init_lr, beta, wd, (1 - i / steps))
            net.zero_grad()
                
            pbar.set_description(f"loss: {loss.item():.4f}")
    
        results[depth].append(loss.item())

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]